In [32]:
import cv2
import numpy as np

def yolo_to_xyxy(box, img_w, img_h):
    x_center, y_center, w, h = box

    x_center *= img_w
    y_center *= img_h
    w *= img_w
    h *= img_h

    x1 = int(x_center - w / 2)
    y1 = int(y_center - h / 2)
    x2 = int(x_center + w / 2)
    y2 = int(y_center + h / 2)

    return x1, y1, x2, y2


In [33]:
def draw_boxes(image, bboxes, labels, class_names):
    img_h, img_w = image.shape[:2]

    for box, label in zip(bboxes, labels):
        x1, y1, x2, y2 = yolo_to_xyxy(box, img_w, img_h)
        class_name = class_names[int(label)]

        # Rectangle
        cv2.rectangle(image, (x1, y1), (x2, y2), (0,255,0), 2)

        # Label
        cv2.putText(image, class_name, (x1, y1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

    return image


In [34]:
def overlay_grid(image, rows=10, cols=10):
    h, w = image.shape[:2]
    # Draw vertical lines
    for i in range(1, cols):
        x = int(i * w / cols)
        cv2.line(image, (x, 0), (x, h), (255, 0, 0), 1)
        cv2.putText(image, f"{i/cols:.2f}", (x+2, 15),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255,0,0), 1)
    # Draw horizontal lines
    for i in range(1, rows):
        y = int(i * h / rows)
        cv2.line(image, (0, y), (w, y), (255, 0, 0), 1)
        cv2.putText(image, f"{i/rows:.2f}", (2, y-2),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255,0,0), 1)
    return image

In [35]:
def test_yolo_label_with_grid(image_path, label_path, class_names, display_width=600):
    image = cv2.imread(image_path)
    h, w = image.shape[:2]
    scale = display_width / w
    new_w = int(w * scale)
    new_h = int(h * scale)
    image = cv2.resize(image, (new_w, new_h))

    # Overlay grid
    image_with_grid = overlay_grid(image.copy(), rows=10, cols=10)

    bboxes = []
    labels = []
    with open(label_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            label = int(parts[0])
            x_c, y_c, w_box, h_box = map(float, parts[1:])
            bboxes.append([x_c, y_c, w_box, h_box])
            labels.append(label)

    result = draw_boxes(image_with_grid, bboxes, labels, class_names)
    cv2.imshow("Label Test with Grid", result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [ ]:
class_names = ['Board', 'Whitefigure', 'Blackfigure', 'Emptyslot']

test_yolo_label_with_grid(
    "dataset/images/board03.jpg",
    "dataset/labels/board03.txt",
    class_names
)